## Cài đặt môi trường và thư viện

In [ ]:
!pip install Bio obonet networkx --quiet

import os
import gc
import random
import numpy as np
import pandas as pd

from Bio import SeqIO
import obonet
import networkx as nx

from tqdm.auto import tqdm

from sklearn.preprocessing import MultiLabelBinarizer

import torch
import torch.nn as nn
from torch import amp
from torch.utils.data import Dataset, DataLoader, random_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Giai đoạn 1: Tối ưu hóa dự đoán bằng GOA UniProt

In [ ]:
FILE_MLP = "/kaggle/input/submission-mlp/submission.tsv"
GOA_FILE = "/kaggle/input/protein-go-annotations/goa_uniprot_all.csv"

graph = obonet.read_obo(
    "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
)
reversed_graph = graph.reverse() # Mặc định OBO là Con->Cha. Cần đảo thành Cha->Con để tìm hậu duệ

descendants_map = {}
for node in tqdm(graph.nodes, desc="Mapping Descendants"):
    descendants_map[node] = nx.descendants(reversed_graph, node) # Tìm con cháu

negative_keys = set()
positive_keys = set()
positive_rows = []

for chunk in tqdm(pd.read_csv(GOA_FILE, usecols=['protein_id', 'go_term', 'qualifier'], chunksize=1000000), desc="Scanning GOA"):
    # Lọc các dòng có NOT
    neg_chunk = chunk[chunk['qualifier'].astype(str).str.contains('NOT', na=False)]

    # zip tránh ghép sai cặp
    for protein_id, go_term in zip(neg_chunk['protein_id'], neg_chunk['go_term']):
        negative_keys.add(f"{protein_id}_{go_term}")

        # Nếu Protein ko có chức năng cha -> ko có chức năng con
        if go_term in descendants_map:
            for desc in descendants_map[go_term]:
                negative_keys.add(f"{protein_id}_{desc}") 

    # Lọc các dòng ko có NOT
    pos_chunk = chunk[~chunk['qualifier'].astype(str).str.contains('NOT', na=False)]
    pos_chunk = pos_chunk.drop_duplicates(subset=['protein_id', 'go_term'])

    for protein_id, go_term in zip(pos_chunk['protein_id'], pos_chunk['go_term']):
        key = f"{protein_id}_{go_term}"
        if key not in positive_keys:
            positive_rows.append(f"{protein_id}\t{go_term}\t1.000")
            positive_keys.add(key)

# Ghi file submission1
with open("submission1.tsv", "w") as f_out:
    # Ghi dữ liệu chuẩn theo UniProt
    if positive_rows:
        f_out.write("\n".join(positive_rows) + "\n")
    # Chống tràn ram
    del positive_rows
    gc.collect()

    # Ghi dữ liệu từ submission-mlp
    with open(FILE_MLP, "r") as f_in:
        for line in tqdm(f_in, desc="Reading MLP"):
            parts = line.strip().split('\t')
            if len(parts) < 3:
                continue

            protein_id, go_term, score = parts[0], parts[1], parts[2]
            key = f"{protein_id}_{go_term}"

            # Nếu protein có NOT hoặc protein đã ghi ở trên rồi thì ko ghi nữa
            if key not in negative_keys and key not in positive_keys:
                f_out.write(line)

print("Done")

## Giai đoạn 2: Xây dựng Mô hình Residual MLP

### Chuẩn bị dữ liệu huấn luyện và test

In [ ]:
INPUT_DIR = "/kaggle/input/cafa-6-protein-function-prediction"
EMBED_DIR = "/kaggle/input/cafa6-protein-embeddings-esm2"

# ids[i] là protein tương ứng với emb[i]
emb = np.load(f"{EMBED_DIR}/protein_embeddings.npy")
ids = pd.read_csv(f"{EMBED_DIR}/protein_ids.csv")["protein_id"].tolist()

# Lấy id của protein trong tập train
train_df = pd.read_csv(f"{INPUT_DIR}/Train/train_terms.tsv", sep="\t")
train_ids = train_df["EntryID"].unique().tolist()

# Lấy id của protein trong tập test
test_ids = []
for protein in SeqIO.parse(f"{INPUT_DIR}/Test/testsuperset.fasta", "fasta"):
    test_ids.append(protein.id.split()[0])

# Lấy index của các protein trong emb
pid_to_index = {pid: i for i, pid in enumerate(ids)}
train_index = [pid_to_index[pid] for pid in train_ids if pid in pid_to_index]
test_index = [pid_to_index[pid] for pid in test_ids if pid in pid_to_index]

# Lấy emb protein của train và test 
X_train = emb[train_index]
X_test = emb[test_index]

# Gom các nhãn GO của từng protein lại thành list
labels = train_df.groupby("EntryID")["term"].apply(list)

# Lấy id của protein vừa có embedding vừa có nhãn GO
protein_ids = [pid for pid in train_ids if pid in labels.index]

# Multi-hot Encoding
mlb = MultiLabelBinarizer()
Y_train = mlb.fit_transform(labels[protein_ids])

### Xây dựng Dataset và mô hình Residual MLP

In [ ]:
class ProteinDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.Y[i]

class ResidualBlock(nn.Module):
    def __init__(self, in_features, hidden_features, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.ReLU(),
            nn.Linear(in_features, hidden_features),
            nn.BatchNorm1d(hidden_features),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_features, in_features) 
        )
        
    def forward(self, x):
        return x + self.block(x) 

class ResNetMLP(nn.Module):
    def __init__(self, input_dim=1280, output_dim=None):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.layer1 = ResidualBlock(2048, 1024, dropout=0.3)
        self.layer2 = ResidualBlock(2048, 1024, dropout=0.3)
        self.layer3 = ResidualBlock(2048, 1024, dropout=0.4)
        self.head = nn.Linear(2048, output_dim)

    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return self.head(x)

### Huấn luyện mô hình 

In [ ]:
# Khóa random seed để kết quả huấn luyện có thể tái lập
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(42)

batch_size = 256
epochs = 18
lr = 2e-3
pos_weight_val = 2.0

dataset = ProteinDataset(X_train, Y_train)
val_size = int(0.1 * len(dataset))
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

# Chia batch, shuffle=True: mỗi epoch sẽ xáo trộn dữ liệu
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2)

# Khởi tạo model
model = ResNetMLP(input_dim=X_train.shape[1], output_dim=Y_train.shape[1]).to(device)

pos_weight = torch.ones([Y_train.shape[1]]).to(device) * pos_weight_val
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3) 
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=5, T_mult=2, eta_min=1e-5
)
scaler = amp.GradScaler(device="cuda")
best_val_loss = float('inf') # khởi tạo với vô cùng

print("Start training...")
for epoch in range(epochs):
    # Huấn luyện
    model.train()
    train_loss = 0
    for Xb, Yb in train_loader:
        Xb, Yb = Xb.to(device), Yb.to(device)
        optimizer.zero_grad()
        with amp.autocast("cuda"):
            logits = model(Xb)
            loss = criterion(logits, Yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * Xb.size(0)
    train_loss /= len(train_loader.dataset)

    # Đánh giá trên tập val
    model.eval()
    val_loss = 0
    with torch.no_grad(): # Không tính gradient
        for Xb, Yb in val_loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            with amp.autocast("cuda"):
                logits = model(Xb)
                loss = criterion(logits, Yb)
            val_loss += loss.item() * Xb.size(0)
    val_loss /= len(val_loader.dataset)

    # Cập nhật learning rate
    scheduler.step(epoch + val_loss/100)
    lr_current = optimizer.param_groups[0]['lr']

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.5f} | Val Loss: {val_loss:.5f} | LR: {lr_current:.1e}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "protein_resnet_best.pt")
        print(f"--> Save Best Model (Val Loss: {val_loss:.5f})")
    else:
        print("--> No improvement.")

print(f"\n Training completed. Best Val Loss: {best_val_loss:.5f}")

### Dự đoán trên tập test

In [ ]:
torch.cuda.empty_cache()

# Khởi tạo mô hình
model = ResNetMLP(input_dim=X_train.shape[1], output_dim=Y_train.shape[1]).to(device)
# Load trọng số từ best model bên trên
model.load_state_dict(torch.load("protein_resnet_best.pt"))
model.eval() 
model = model.half()

test_id_to_index = {pid: i for i, pid in enumerate(test_ids)}
all_go_terms = mlb.classes_
batch_size = 32
min_prob = 0.01
top_k = 25

submission_rows = []
for i in tqdm(range(0, len(test_ids), batch_size), desc="Predicting..."):
    batch_ids = test_ids[i:i+batch_size]
    batch_idx = [test_id_to_index[pid] for pid in batch_ids]
    batch = X_test[batch_idx].astype(np.float16)
    batch = torch.tensor(batch, dtype=torch.float16).to(device)

    # Tính xác suất dự đoán cho mỗi nhãn GO
    with torch.no_grad():
        logits = model(batch)
        probs = torch.sigmoid(logits).cpu().numpy()

    for j, pid in enumerate(batch_ids):
        p = probs[j]
        idx = np.where(p >= min_prob)[0]
        if top_k is not None and len(idx) < top_k:
            top_extra = p.argsort()[-top_k:]
            idx = np.unique(np.concatenate([idx, top_extra]))
        idx = idx[np.argsort(-p[idx])]
        if top_k is not None:
             idx = idx[:top_k]

        for k in idx:
            term = all_go_terms[k]
            score = float(p[k])
            submission_rows.append([pid, term, score])

    del batch, logits, probs
    torch.cuda.empty_cache()

# Ghi file submission2
submission_df = pd.DataFrame(submission_rows, columns=["EntryID", "GO", "Score"])
submission_df.to_csv("submission2.tsv", sep="\t", index=False, header=False)
print("Done")

## Giai đoạn 3: Ensemble

In [ ]:
WEIGHT_A = 0.7
WEIGHT_B = 0.3
DIVERSITY_BONUS = 0.05
THRESHOLD = 0.01
TOP_K = 30
FILE_A = "/kaggle/working/submission1.tsv"
FILE_B = "/kaggle/working/submission2.tsv"

# Lấy các go terms hợp lệ
train_df = pd.read_csv(f"{INPUT_DIR}/Train/train_terms.tsv", sep="\t", usecols=["term"])
valid_terms = set(train_df["term"].unique())

preds = {}
print("Load file A...")
with open(FILE_A, 'r') as f:
    for line in tqdm(f):
        parts = line.strip().split('\t')
        if len(parts) < 3: continue       
        if parts[1] not in valid_terms: continue         
        key = f"{parts[0]}\t{parts[1]}"
        preds[key] = {'a': float(parts[2]), 'b': 0.0}
        
print("Loading file B...")
with open(FILE_B, 'r') as f:
    for line in tqdm(f):
        parts = line.strip().split('\t')
        if len(parts) < 3: continue  
        if parts[1] not in valid_terms: continue    
        key = f"{parts[0]}\t{parts[1]}"   
        if key in preds: preds[key]['b'] = float(parts[2])
        else: preds[key] = {'a': 0.0, 'b': float(parts[2])}

final_data = []
for key, scores in tqdm(preds.items(), desc="Computing..."):
    score_a = scores['a']
    score_b = scores['b']
    final_score = (score_a * WEIGHT_A) + (score_b * WEIGHT_B)    
    if score_a > 0.01 and score_b > 0.01:
        final_score += DIVERSITY_BONUS # Thêm điểm bonus nếu cả hai đều dự đoán xác suất > 0.01
    final_score = min(final_score, 1.0)   
    if final_score >= THRESHOLD:
        pid, term = key.split('\t')
        final_data.append([pid, term, final_score])
del preds

# Ghi file submission3
df = pd.DataFrame(final_data, columns=["Id", "Term", "Score"])
df = df.sort_values(by=["Id", "Score"], ascending=[True, False])
df_clean = df.groupby("Id").head(TOP_K).copy()
df_clean["Score"] = df_clean["Score"].round(3)
df_clean.to_csv("submission3.tsv", sep='\t', index=False, header=False)

print("Done")

## Link tải file submission của 3 giai đoạn

In [ ]:
from IPython.display import FileLink

print("Giai đoạn 1 (0.32):")
display(FileLink("submission1.tsv"))
print("Giai đoạn 2 (0.295):")
display(FileLink("submission2.tsv"))
print("Giai đoạn 3 (0.342):")
display(FileLink("submission3.tsv"))